In [3]:
# load the model from the file "sbm3x3_pclam_roc_0.210_auc_0.860"
import torch
from torch_geometric.transforms import TwoHop

import sys
if '..' not in sys.path:
    sys.path.insert(0, '..')

from datasets.import_dataset import import_dataset
from utils.plotting import *
from trainer import Trainer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using device:', device)
cpu = torch.device("cpu")

%load_ext autoreload
%autoreload 2


Using device: cuda
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:

model_names = ['iegam','piegam', 'bigclam', 'pclam']
ds_names = ['ellipticGGAD', 'redditGGAD', 'photoGGAD']
#! problem with something to do with attr with reddit and elliptic
for ds_name in ds_names:    
    ds = import_dataset(ds_name)
    fat_ds = TwoHop()(ds)
    fat_ds.edge_attr = torch.ones(fat_ds.edge_index.shape[1]).bool()

    if ds_name in ['Flickr', 'ACM', 'BlogCatalog']:
        ds_to_use = ds
    elif ds_name in ['redditGGAD', 'photoGGAD', 'ellipticGGAD']:
        ds_to_use = fat_ds
    else:
        raise ValueError('ds_name not recognized')


    losseses = []
    acc_testses = []
    acc_valses = []
    
    for model_name in model_names:
        config_triplets = [
            ['clamiter_init', 'dim_feat', 30],
            ['clamiter_init', 'dim_attr', 100],
            # ['feat_opt','n_iter', 100],
            # ['prior_opt', 'n_iter', 200],
            ['feat_opt', 'lr', 3e-6],
            ['prior_opt', 'lr', 2e-6],
            ['prior_opt', 'noise_amp', 0.05],
            ['back_forth', 'n_back_forth', 5],
            ['back_forth', 'scheduler_step_size', 1],
            ['back_forth', 'scheduler_gamma', 0.5],
            ['back_forth', 'first_func_in_fit', first_func]
        ]



        trainer_anomaly = Trainer(
            model_name=model_name,
            device=device,
            dataset=ds_to_use.clone(),
            attr_opt=True,
            task='anomaly',
            mighty_configs_dict=True,
            config_triplets_to_change=config_triplets
        )

        losses, acc_test, acc_val = trainer_anomaly.train(
            init_type='small_gaus',
            init_feats=True,
            acc_every=20,
            plot_every=-1,
            verbose=True,
            verbose_in_funcs=False
        )
        losseses.append(losses)
        acc_testses.append(acc_test)
        acc_valses.append(acc_val)

del ds, fat_ds





/home/user/Documents/danny/AAAI_pieclam/tests/../trainer.py:415:::  
 starting optimization of piegam on ellipticGGAD on device cuda

 configs_dict: 
{
    "clamiter_init": {
        "dim_feat": 30,
        "dim_attr": 93,
        "s_reg": 0.0,
        "l1_reg": 1,
        "T": 1,
        "hidden_dim": 64,
        "num_coupling_blocks": 32,
        "num_layers_mlp": 2
    },
    "feat_opt": {
        "lr": 3e-06,
        "n_iter": 500,
        "early_stop": 0
    },
    "prior_opt": {
        "n_iter": 1300,
        "lr": 2e-06,
        "noise_amp": 0.05,
        "weight_decay": 0.1,
        "early_stop": 0
    },
    "back_forth": {
        "n_back_forth": 5,
        "scheduler_step_size": 1,
        "scheduler_gamma": 0.5,
        "early_stop_fit": 0,
        "first_func_in_fit": "fit_feats"
    }
}


/home/user/Documents/danny/AAAI_pieclam/tests/../trainer.py:420:::  
 train_model_on_params, initializing feats with small_gaus


/home/user/Documents/danny/AAAI_pieclam/tests/../trai

KeyboardInterrupt: 